In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package 

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/content/drive/MyDrive/HAI research/output/{today}/py_output").mkdir(parents=True, exist_ok=True)

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI research/data/PoC 평가.xlsx", sheet_name='total')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) : 
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[8] 

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
userResponse

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI research/data/PoC_Question.csv")

In [ ]:
def PoC_CoT_test(q_1:str, q_2:str, q_3:str, plot:list, ChatGPT_api_key:str, data_path:str, model:str="gpt-3.5-turbo")->str:

    openai.api_key = ChatGPT_api_key

    messages = [
        {
            "role": "system",
            "content": "인공지능 모델을 활용하여 개선이 가능한 영역인지 확인해줘",
        },
        
        {
            "role": "system",
            "content": "인공지능을 활용했을 때, 회귀, 분류 모델 중 어떤 모델이 적합한지 확인해줘",
        },
        
        {
            "role": "system",
            "content": "개선이 필요한 영역을 한 줄로 정리해서 요약해줘",
        },
        
        {
            "role": "user", 
            "content": f'인공지능을 적용하여여 개선이 필요한 영역은 "{q_1}"이야'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)

    answer_1 = response["choices"][0]["message"]["content"]
    
    messages = [
        {
            "role": "system",
            "content": f"'{answer_1}' 영역에 대해 인공지능을 활용하고 싶어",
        },
        
        {
            "role": "system",
            "content": "인공지능이 예측하려는 Target이 무엇인지 확인해줘",
        },
        
        {
            "role": "system",
            "content": "인공지능의 목적을 한 줄로 정리해서 요약해줘.",
        },

        {
            "role": "user", 
            "content": f'인공지능의 목적은 "{q_2}"이야'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_2 = response["choices"][0]["message"]["content"]
    
    messages = [
        {
            "role": "system",
            "content": f"AI의 주제는 '{answer_2}'이야",
        },
     
        {
            "role": "system",
            "content":"python script를 사용하여 dataframe을 생성해줘",
        },

        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

        {
            "role": "system",
            "content": "입력된 데이터를 feature와 label 두 종류로 구분해줘",
        },

        {
            "role": "system",
            "content": "dataframe 생성 시, column의 명칭은 입력된 feature와 Label 명칭으로 사용해줘",
        },
        
        {
            "role": "system",
            "content": "dataframe의 value값들을 임의로 100개만 생성해줘",
        },    

        {
            "role": "system",
            "content": "임의로 생성하는 dataframe의 value값들은 feature의 특성에 맞게 생성해줘",
        },    

         {
            "role": "system",
            "content": f"생성한 dataframe의 저장 경로는 {data_path}야",
        },   
         {
            "role": "system",
            "content": f"dataframe은 csv 파일 형태로 저장해주고 저장하는 파일의 명칭은 {userCode}이야",
        },  
         {
            "role": "system",
            "content": "위 단계들을 차근차근 생각해줘",
        },   
        {
            "role": "user", 
            "content": f"데이터 세트는 '{q_3}'이야. 이를 참고하여 dataframe을 생성하는 python script를 만들어줘"
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_3 = response["choices"][0]["message"]["content"]

    messages = [
        {
            "role": "system",
            "content": f"지금부터 {answer_3}를 리뷰할거야",
        },
        {
            "role": "system",
            "content": "차근차근 생각하고 효율적으로 코드를 구축해줘",
        },
        
        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

         {
            "role": "system",
            "content":f"생성한 {answer_3}가 데이터프레임을 지정된 경로에 잘 생성하는지 체크해줘",
        }, 

         {
            "role": "system",
            "content":f"생성한 {answer_3}에 {q_3}을 참고하여 feature와 Label 이름이 잘 부여되었는지 체크해줘",
        },   

        {
            "role": "system",
            "content": f"실행하는데 문제가 발생하지 않을 경우 {answer_3} python script를 그대로 출력해줘",
        },

        {
            "role": "user", 
            "content": f'생성한 {answer_3}를 리뷰해주고 python script를 전달해줘'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_3 = response["choices"][0]["message"]["content"]

    messages = [
            {
                "role": "system",
                "content": f"인공지능의 목적은 '{answer_2}'이야",
            },
            {
                "role": "system",
                "content": f"현재 보유한 데이터프레임은 {answer_3}이야",
            },
            
            {
                "role": "system",
                "content": "AI Modeling 관점으로 사용가능한 plot들을 1개만 추천해줘",
            },

            {
                "role": "system",
                "content":"데이터프레임의 feature들을 참고하여 사용가능한 plot들을 1개만 추천해줘",
            },
        
            {
                "role": "system",
                "content": f"plot들을 추천할 시 {plot} 리스트 목록에 존재하는 plot들만 추천해줘",
            },

            {
                "role": "user", 
                "content": '데이터프레임을 참고하여 필요한 시각화 plot들을 추천해줘'
            }
        ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_4 = response["choices"][0]["message"]["content"]

    pprint(answer_1)
    pprint(answer_2)

    return answer_3, answer_4

In [ ]:
output, output_2 = PoC_CoT_test(q_1=userResponse[5], q_2=userResponse[8], q_3=userResponse[22],
                      plot = ["feature_importance", "roc_curve", "bar_plot", "line_plot", "pie_plot", "heatmap", "confusion_matrix", "histogram"],
                      ChatGPT_api_key=openai.api_key, 
                      data_path=f'/content/drive/MyDrive/HAI research/output/{today}/py_output',
                      model="gpt-3.5-turbo")

In [ ]:
pprint(output)

In [ ]:
# 일단 대충 얼기설기 짜놓았습니다.
# 앞에서부터 탐색 / 뒤에서부터 탐색 두개로 각각 발견되면 멈추게 짜면 될 것 같습니다.
divided = output.split('\n')
idx = []
for i, w in enumerate(divided):
    if w.startswith("```") or w.endswith("```"):
        idx.append(i)
output_code = '\n'.join(divided[idx[0]+1:idx[1]])

pprint(output_code)

In [ ]:
with open(f'/content/drive/MyDrive/HAI research/output/{today}/output_poc_{userCode}.py', 'w') as f:
    f.write(output_code)

In [ ]:
%run -i '/content/drive/MyDrive/HAI research/output/{today}/output_poc_{userCode}.py'

In [ ]:
# 발생한 주요 이슈
# 1. 리뷰 shot을 작성했지만 token 초과 이슈가 지속적으로 발생(토큰 이슈 때문에 프롬프트 작성에 한계가 있음)
# 2. run 할 때마다의 python script 및 데이터프레임이 계속 바뀜
# ex) 프레임을 생성 시, random 하게 뽑아내는 범위가 각각 다름, 프롬프트로 지시하였지만 feature와 label명을 가져오지 못하는 경우도 발생
# 3. data_path가 한국어->영어->한국어로 번역되는 과정에서 경로가 바뀜 -> 경로는 영어로 세팅해야할 것으로 판단 -> 폴더 명칭 HAI 연구개발 리서치 -> HAI research로 변경
# 4. 22번 문항이 없고 28번 문항만 있을 경우 어떻게 처리해야할지? / 22번, 28번 둘 다 작성한 인원의 경우 어떻게 처리해야할지?